In [1]:
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

from nemo.global_planner import AStarGradPlanner
from nemo.nemo import Nemo
from nemo.util import wrap_angle_torch, path_metrics
from nemo.plotting import plot_surface, plot_path_3d
from nemo.planning import path_optimization

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%load_ext autoreload
%autoreload 2